## Advent of Code - Day 19

In [1]:
import numpy as np
import re
from scipy.spatial.distance import pdist
from collections import Counter

### Star 1

In [2]:
def load_data(fn):
    p = re.compile(r'[\d-]+')
    scanners = []
    with open(fn) as f:
        for line in f:
            if 'scanner' in line:
                scanner = []
                continue
            if len(line) < 5:
                scanners.append(scanner)
                continue
            scanner.append([int(x) for x in p.findall(line)])
    scanners.append(scanner)
    
    return scanners

In [3]:
def find_pairs(scanners):
    '''If 2 scanners have >60 matching pair-wise beacon distances, then overlapping.'''
    signatures = []
    for scanner in scanners:  # calc all pair-wise distances within each scanner
        signatures.append(set(np.round(pdist(scanner), decimals=2)))
    pairs = set()
    for i in range(len(signatures)):
        for j in range(i + 1, len(signatures)):
            intersection = len(signatures[i].intersection(signatures[j]))
            if intersection > 60:
                pairs.add((i, j))
                
    return pairs

In [4]:
def gen_all_rots():
    rot_z = np.array([[0, 1, 0],  # rotate around z-axis (4 rotations)
                     [-1, 0, 0],
                    [0, 0, 1]])
    rot_x = np.array([[1, 0, 0],  # rotate around x-axis (4 rotations)
                     [0, 0, -1],
                     [0, 1, 0]])
    rot_y = np.array([[0, 0, -1], # rotate around y-axis (2 rotations)
                     [0, 1, 0],
                     [1, 0, 0]])
# build 4 z-rotations
    rot1 = []
    mat = np.eye(3)
    for i in range(4):   
        mat = np.matmul(mat, rot_z)
        rot1.append(mat)
# build 4 x-rotations + 2 y-rotations
    rot2 = []
    for i in range(4):
        mat = np.matmul(mat, rot_x)
        rot2.append(mat)
    mat = np.matmul(mat, rot_y)  # rotate 90deg around y-axis
    rot2.append(mat)
    mat = np.matmul(mat, rot_y)
    mat = np.matmul(mat, rot_y)  # rotate 2x90deg around y-axis
    rot2.append(mat)
# build 24 (4 x 6) combinations
    return [np.matmul(x, y) for x in rot1 for y in rot2]

In [5]:
def de_dup(b1, b2):
    stack = np.vstack((b1, b2))
    s = {tuple(x) for x in stack}
    
    return np.array([x for x in s], dtype=np.int32)
    
def align(scan1, scan2, all_rots):
    for rot in all_rots:
        c = Counter()
        b1 = scan1
        b2 = scan2
        b2 = np.matmul(b2, rot).astype(np.int32)
        [c.update((tuple(x-y),)) for x in b1 for y in b2]
        if c.most_common(1)[0][1] >= 12:
            break
    shift = c.most_common(1)[0][0]
    b2 += np.array(shift)
    
    return (de_dup(b1, b2), shift)

In [6]:
scanners = load_data('input_19_a.txt')
pairs = find_pairs(scanners)
all_rots = gen_all_rots()
stack = [0]
shifts = []
all_beacons = np.array(scanners[0], dtype=np.int32)  # start with scanner
while len(pairs) > 0:
    next_scanner = stack.pop()
    remove_pairs = []
    for pair in pairs:
        if next_scanner in pair:
            s = list(pair)
            s.remove(next_scanner)
            stack.append(s[0])
            scanner = np.array(scanners[s[0]], dtype=np.int32)
            all_beacons, shift = align(all_beacons, scanner, all_rots)
            shifts.append(shift)  # keep track of all shifts for star 2
            remove_pairs.append(pair)
    for pair in remove_pairs:
        pairs.remove(pair)
        
print(f'Number of beacons = {all_beacons.shape[0]}')

Number of beacons = 459


### Star 2

In [7]:
shifts = np.array([np.array(x) for x in shifts])
mx = int(pdist(shifts, metric='cityblock').max())

print(f'Maximum Manhattan distance between scanners = {mx}')

Maximum Manhattan distance between scanners = 19130
